In [ ]:
import tarfile

In [ ]:
my_tar = tarfile.open('/content/drive/MyDrive/yelp_dataset.tar')

In [ ]:
my_tar.extractall('./yelp_dataset')

In [ ]:
my_tar.close()

In [ ]:
%run -i '/content/j_csv.py' '/content/yelp_dataset/yelp_academic_dataset_business.json'

In [ ]:
%run -i '/content/j_csv.py' '/content/yelp_dataset/yelp_academic_dataset_checkin.json'

In [ ]:
%run -i '/content/j_csv.py' '/content/yelp_dataset/yelp_academic_dataset_user.json'

In [ ]:
%run -i '/content/j_csv.py' '/content/yelp_dataset/yelp_academic_dataset_tip.json'

In [ ]:
%run -i '/content/j_csv.py' '/content/yelp_dataset/yelp_academic_dataset_review.json'

# DATA PREPAREATION AND CLEANING

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# reading csv files
data_tip=pd.read_csv('/content/yelp_dataset/yelp_academic_dataset_tip.csv')


In [ ]:
data_tip.head()

In [ ]:
data_tip.describe()

In [ ]:
data_tip.columns

In [ ]:
data_tip.info()

In [ ]:
# drop entries with 'NaN' under 'text' column
data_tip = data_tip[data_tip.text.notnull()]
data_tip.reset_index(inplace=True, drop=True)

print(data_tip.info())

In [ ]:
data_tip.to_csv('tip_final.csv',index=False)

# checkin data

In [ ]:
data_checkin=pd.read_csv('/content/yelp_dataset/yelp_academic_dataset_checkin.csv')

In [ ]:
data_checkin.info()

In [ ]:
# sort the columns by column names
ordered_names = sorted(data_checkin.columns)
checkin = data_checkin[ordered_names]
print(data_checkin.columns)

# add a new column named 'total_count' containing the sum of all checkins at all times (sum of all values under the nested columns with names starting with 'time.')
checkin['total_count'] = data_checkin.loc[:,'time.Fri-0':'time.Wed-9'].sum(axis=1).astype(int)

print(checkin.info())

In [ ]:
data_checkin.to_csv('checkin_final.csv',index=False)

# user data

In [ ]:
data_user=pd.read_csv('/content/yelp_dataset/yelp_academic_dataset_user.csv')

In [ ]:
data_user.head()

In [ ]:
data_user.describe()

In [ ]:
data_user.columns

In [ ]:
#drop data if average _stars =0 it should be between 1 to 5
print(data_user.average_stars.describe())


In [ ]:
data_user = data_user[data_user.average_stars != 0]
data_user.reset_index(inplace=True, drop=True)

In [ ]:
data_user.to_csv('user_final.csv',index=False)

# review data

In [ ]:
data=pd.read_csv('/content/yelp_dataset/yelp_academic_dataset_business.csv')

In [ ]:
ziplen = data.postal_code.astype(str).apply(len) # drop entries where postal_code is not 5-digit from business
business = data[ziplen == 5]

list_of_states = ['AL','AK','AR','AS','AZ','CA','CO','CT','DC','DE','FL','FM','GA','GU','HI','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME','MH','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','MP','OH','OK','OR','PA','PR','PW','RI','SC','SD','TN','TX','UT','VA','VI','VT','WA','WI','WV','WY']
business = business[business.state.isin(list_of_states)]

In [ ]:
# deal with missing city information
data.loc[58688,'city'] = 'Yuma' # add missing city info
data.loc[111036,'city'] = 'Raleigh' # add missing city info

# remove one entry with missing latitude information
data = data[data.latitude.notnull()]

In [ ]:
# business category

from collections import defaultdict

category = defaultdict(int)

for idx, row in data.iterrows():
    if row['categories'] is not np.nan: 
        entries = row['categories'].split(',')
        for entry in entries: 
            entry = entry.strip().lower()
            category[entry] += 1

print(len(category))
sorted_cate = sorted(category.items(), key=lambda x: x[1], reverse=True)
print(sorted_cate[:10])

In [ ]:
"""
    remove businesses other than restaurants based on the information provided under business 'categories'
    """
def rmv_non_restaurant_busi(df, yes=[], no=[]): 
    
  

    yes = set(['fast food','sandwiches','caterers','deserts','burgers']).union(set(yes))
    no = set(['shopping','home services','beauty & spas','hair salons','health & medical', 'local services','automotive',
              'fitness & instruction','transportation','airlines','pets','active life','professional services','banks & credit unions'
             'insurance','financial services','education','public services & government','session photography','car rental',
             'photographers','churches','religious organizations','tours','doctors','massage','pharmacy','drugstores']).union(set(no))
    idx_to_drop = []
    for idx, row in df.iterrows():
        if row['categories'] is not np.nan: 
            entries = row['categories'].split(',')
            for entry in entries:
                entry = entry.strip().lower()
                if entry in yes: 
                    break
                if entry in no:
                    idx_to_drop.append(idx)
                    break
        elif row['attributes.HairSpecializesIn'] is not np.nan:  # if 'categories' is not available, use 'attributes' to filter
            idx_to_drop.append(idx) 
                    
    return df.drop(index=idx_to_drop)

print("# of businesses of all categories:", len(business))
business = rmv_non_restaurant_busi(business)
print("# of businesses related to restaurants:", len(business))

In [ ]:
# extract cuisine and style information from the 'categories' columns

# define a default list for restaurant cuisines and restaurant styles
cuisines = ['mexican','italian','chinese','japanese','thai','indian','american (new)','american (traditional)',\
            'french','middle eastern','korean','mediterranean','vietnamese','cajun','greek','hawaiian',\
            'asian fusion','vegetarian','vegan','steakhouse','barbeque','sushi bars','tex-mex','specialty food',\
            'gluten-free','coffee & tea','desserts','seafood','ice cream & frozen yogurt','bakeries','beer',\
            'wine & spirits','soup','pizza','hot dogs','burgers','donuts','cupcakes','salad','tacos',\
            'chicken wings','sandwiches','bubble tea','tapas/small plates','shaved ice','bagels','southern',\
            'local flavor','latin american','custom cakes','ethinic food']
styles = ['restaurants','fast food','food stands','street vendors','nightlife','buffets','bars','food trucks',\
          'breakfast & brunch','diners','cocktail bars','pubs','sports bars','wine bars','beer bars',\
          'casinos','juice bars & smoothies','caterers','delis','cafes','lounges','music venues',\
          'performing arts','food delivery services','dive bars','dance clubs','breweries']

# initiate two new columns 'cuisine' and 'style'
data['cuisine'] = None
data['style'] = None

# loop over the 'data' dataframe and extract cuisines and styles from the 'categories' column
for idx, row in data.iterrows():
    if row['categories'] is not np.nan:
        entries = row['categories'].split(',')
        for entry in entries:
            entry = entry.strip().lower()
            if entry in cuisines: 
                if data.loc[idx,'cuisine'] == None: 
                    data.loc[idx,'cuisine'] = entry
                else:
                    data.loc[idx,'cuisine'] += (', ' + entry)
                    
            elif entry in styles:
                if data.loc[idx,'style'] == None:
                    data.loc[idx,'style'] = entry
                else:
                   data.loc[idx,'style'] += (', ' + entry)
data.fillna(value=pd.np.nan, inplace=True) # update None with NaN

In [ ]:
data.reset_index(drop=True, inplace=True)
print(data.info())

In [ ]:
data.to_csv('business_final.csv',index=False)